**[Microchallenges Home Page](https://www.kaggle.com/learn/microchallenges)**

---


# Intro

Data scientists tend to focus on **prediction** because that's where conventional machine learning excels. But real world decision-making involves both prediction and **optimization**.  After predicting what will happen, you decide what to do about it.

Optimization gets less attention than it deserves. So this micro-challenge will test your optimization skills as you write a function to improve how airlines set prices.

![Imgur](https://i.imgur.com/AKrbLMR.jpg)


# The Problem

You recently started Aviato.com, a startup that helps airlines set ticket prices. 

Aviato's success will depend on a function called `pricing_function`.  This notebook already includes a very simple version of `pricing_function`.  You will modify `pricing_function` to maximize the total revenue collected for all flights in our simulated environment.

For each flight, `pricing_function` will be run once per (simulated) day to set that day's ticket price. The seats you don't sell today will be available to sell tomorrow, unless the flight leaves that day.

Your `pricing_function` is run for one flight at a time, and it takes following inputs:
- **Number of days until the flight**
- **Number of seats they have left to sell**
- **A variable called `demand_level` that determines how many tickets you can sell at any given price. **

The quantity you sell at any price is:
> quantity_sold = demand_level - price

Ticket quantities are capped at the number of seats available.

Your function will output the ticket price.

You learn the `demand_level` for each day at the time you need to make predictions for that day. For all days in the future, you only know `demand_level` will be drawn from the uniform distribution between 100 and 200.  So, for any day in the future, it is equally likely to be each value between 100 and 200.

In case this is still unclear, some relevant implementation code is shown below.

# The Simulator
We will run your pricing function in a simulator to test how well it performs on a range of flight situations.  **Run the following code cell to set up your simulation environment:**

In [1]:
import sys
sys.path.append('../input')
from flight_revenue_simulator import simulate_revenue, score_me


In case you want to check your understanding of the simulator logic, here is a simplified version of some of the key logic (leaving out the code that prints your progress). If you feel you understand the description above, you can skip reading this code.

```
def _tickets_sold(p, demand_level, max_qty):
        quantity_demanded = floor(max(0, p - demand_level))
        return min(quantity_demanded, max_qty)

def simulate_revenue(days_left, tickets_left, pricing_function, rev_to_date=0, demand_level_min=100, demand_level_max=200):
    if (days_left == 0) or (tickets_left == 0):
        return rev_to_date
    else:
        demand_level = uniform(demand_level_min, demand_level_max)
        p = pricing_function(days_left, tickets_left, demand_level)
        q = _tickets_sold(demand_level, p, tickets_left)
        return _total_revenue(days_left = days_left-1, 
                              tickets_left = tickets_left-q, 
                              pricing_function = pricing_function, 
                              rev_to_date = rev_to_date + p * q,
                              demand_level_min = demand_level_min,
                              demand_level_max = demand_level_max
                             )
```

# Your Code

Here is starter code for the pricing function.  If you use this function, you will sell 10 tickets each day (until you run out of tickets).

In [13]:
def pricing_function(days_left, tickets_left, demand_level):
    """Sample pricing function"""
    price = demand_level - 9
    return price

In [19]:
import numpy as np
from scipy.optimize import minimize

In [27]:
x0 = np.array([0, 0, 1, -9])


def cal(x):
    x = x
    def pricing_function(days_left, tickets_left, demand_level):
        """Sample pricing function"""
        price = x[3] + x[2]*demand_level - x[1]*tickets_left - x[0]*days_left
        return price
    sim = simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)
    val = 1/sim
    return val

x = minimize(cal, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})
x.x

7 days before flight: Started with 50 seats. Demand level: 192. Price set to $183. Sold 9 tickets. Daily revenue is 1645. Total revenue-to-date is 1645. 41 seats remaining
6 days before flight: Started with 41 seats. Demand level: 104. Price set to $95. Sold 9 tickets. Daily revenue is 855. Total revenue-to-date is 2500. 32 seats remaining
5 days before flight: Started with 32 seats. Demand level: 111. Price set to $102. Sold 9 tickets. Daily revenue is 916. Total revenue-to-date is 3416. 23 seats remaining
4 days before flight: Started with 23 seats. Demand level: 115. Price set to $106. Sold 9 tickets. Daily revenue is 957. Total revenue-to-date is 4373. 14 seats remaining
3 days before flight: Started with 14 seats. Demand level: 137. Price set to $128. Sold 9 tickets. Daily revenue is 1150. Total revenue-to-date is 5523. 5 seats remaining
2 days before flight: Started with 5 seats. Demand level: 128. Price set to $119. Sold 5 tickets. Daily revenue is 597. Total revenue-to-date is 

4 days before flight: Started with 17 seats. Demand level: 162. Price set to $150. Sold 11 tickets. Daily revenue is 1651. Total revenue-to-date is 5924. 6 seats remaining
3 days before flight: Started with 6 seats. Demand level: 193. Price set to $181. Sold 6 tickets. Daily revenue is 1086. Total revenue-to-date is 7010. 0 seats remaining
This flight is booked full.
Total Revenue: $7010
7 days before flight: Started with 50 seats. Demand level: 120. Price set to $109. Sold 11 tickets. Daily revenue is 1200. Total revenue-to-date is 1200. 39 seats remaining
6 days before flight: Started with 39 seats. Demand level: 179. Price set to $167. Sold 11 tickets. Daily revenue is 1834. Total revenue-to-date is 3034. 28 seats remaining
5 days before flight: Started with 28 seats. Demand level: 111. Price set to $100. Sold 10 tickets. Daily revenue is 998. Total revenue-to-date is 4032. 18 seats remaining
4 days before flight: Started with 18 seats. Demand level: 159. Price set to $147. Sold 11 

array([ 1.68380107e-04,  1.35435183e-04,  9.85113660e-01, -9.21853046e+00])

To see a small example of how your code works, test it with the following function:

In [29]:
y = x.x.tolist()
y

[0.000168380106694876,
 0.00013543518336928882,
 0.9851136599793421,
 -9.218530457881501]

In [30]:
def pricing_function_1(days_left, tickets_left, demand_level):
        """Sample pricing function"""
        price = y[3] + y[2]*demand_level - y[1]*tickets_left - y[0]*days_left
        return price

In [16]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)

7 days before flight: Started with 50 seats. Demand level: 134. Price set to $125. Sold 9 tickets. Daily revenue is 1127. Total revenue-to-date is 1127. 41 seats remaining
6 days before flight: Started with 41 seats. Demand level: 148. Price set to $139. Sold 9 tickets. Daily revenue is 1250. Total revenue-to-date is 2377. 32 seats remaining
5 days before flight: Started with 32 seats. Demand level: 195. Price set to $186. Sold 9 tickets. Daily revenue is 1670. Total revenue-to-date is 4047. 23 seats remaining
4 days before flight: Started with 23 seats. Demand level: 190. Price set to $181. Sold 9 tickets. Daily revenue is 1627. Total revenue-to-date is 5674. 14 seats remaining
3 days before flight: Started with 14 seats. Demand level: 142. Price set to $133. Sold 9 tickets. Daily revenue is 1200. Total revenue-to-date is 6874. 5 seats remaining
2 days before flight: Started with 5 seats. Demand level: 110. Price set to $101. Sold 5 tickets. Daily revenue is 503. Total revenue-to-date

7376.955182666899

You can try simulations for a variety of values.

Once you feel good about your pricing function, run it with the following cell to to see how it performs on a wider range of flights.

In [18]:
score_me(pricing_function_1)

Ran 200 flights starting 100 days before flight with 100 tickets. Average revenue: $14143
Ran 200 flights starting 14 days before flight with 50 tickets. Average revenue: $7066
Ran 200 flights starting 2 days before flight with 20 tickets. Average revenue: $2469
Ran 200 flights starting 1 days before flight with 3 tickets. Average revenue: $420


<IPython.core.display.Javascript object>

Average revenue across all flights is $6025


---
**[Microchallenges Home Page](https://www.kaggle.com/learn/microchallenges)**


*Want to discuss your solution or hear what others have done?  Visit the [Learn Discussion forum](https://www.kaggle.com/general/62469) to chat with other Learners.*